# Portfolio Optimization using Second Order Cone

In this notebook we show how to use the Second Order Cone (SOC) constraint in the variance portfolio optimization problem.

## 1. Variance Optimization

### 1.2 Variance Minimization

The minimization of portfolio variance is a quadratic optimization problem and can be posed as:

$$
\begin{equation}
\begin{aligned}
& \underset{x}{\text{min}} & &  x^{\tau} \Sigma x \\
& \text{s.t.} & & \mu x^{\tau} \geq \bar{\mu} \\
& & &  \sum_{i=1}^{N} x_i = 1 \\
& & &  x_i \geq 0 \; ; \; \forall \; i =1, \ldots, N \\
\end{aligned}
\end{equation}
$$

Where $x$ are the weights of assets, $\mu$ is the mean vector of expected returns, $\bar{\mu}$ the minimum expected return of portfolio and $r$ is the matrix of observed returns.

In [1]:
####################################
# Downloading Data
####################################

import numpy as np
import pandas as pd
import yfinance as yf
import warnings

warnings.filterwarnings("ignore")

yf.pdr_override()
pd.options.display.float_format = '{:.4%}'.format

# Date range
start = '2016-01-01'
end = '2019-12-30'

# Tickers of assets
assets = ['JCI', 'TGT', 'CMCSA', 'CPB', 'MO', 'APA', 'MMC', 'JPM',
          'ZION', 'PSA', 'BAX', 'BMY', 'LUV', 'PCAR', 'TXT', 'TMO',
          'DE', 'MSFT', 'HPQ', 'SEE', 'VZ', 'CNP', 'NI', 'T', 'BA']
assets.sort()

# Downloading data
data = yf.download(assets, start = start, end = end)
data = data.loc[:,('Adj Close', slice(None))]
data.columns = assets

# Calculating returns
Y = data[assets].pct_change().dropna()

display(Y.head())

[*********************100%***********************]  25 of 25 completed


,APA,BA,BAX,BMY,CMCSA,CNP,CPB,DE,HPQ,JCI,...,NI,PCAR,PSA,SEE,T,TGT,TMO,TXT,VZ,ZION
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-05,-2.0257%,0.4057%,0.4036%,1.9693%,0.0180%,0.9305%,0.3678%,0.5783%,0.9483%,-1.1953%,...,1.5881%,0.0212%,2.8236%,0.9758%,0.6987%,1.7539%,-0.1730%,0.2410%,1.3735%,-1.0857%
2016-01-06,-11.4863%,-1.5879%,0.2412%,-1.7556%,-0.7727%,-1.2473%,-0.1736%,-1.1239%,-3.5867%,-0.9551%,...,0.5547%,0.0212%,0.1592%,-1.5647%,-0.1466%,-1.0155%,-0.7653%,-3.0048%,-0.9034%,-2.9145%
2016-01-07,-5.1389%,-4.1922%,-1.6573%,-2.7699%,-1.1047%,-1.9769%,-1.2207%,-0.8855%,-4.6058%,-2.5394%,...,-2.2066%,-3.0309%,-1.0411%,-3.1557%,-1.6148%,-0.2700%,-2.2845%,-2.0570%,-0.5492%,-3.0019%
2016-01-08,0.2736%,-2.2705%,-1.6037%,-2.5425%,0.1099%,-0.2241%,0.5707%,-1.6402%,-1.7642%,-0.1649%,...,-0.1539%,-1.1366%,-0.7308%,-0.1448%,0.0895%,-3.3838%,-0.1117%,-1.1387%,-0.9719%,-1.1254%
2016-01-11,-4.3383%,0.1692%,-1.6851%,-1.0215%,0.0915%,-1.1791%,0.5674%,0.5287%,0.6616%,0.0330%,...,1.6436%,0.0000%,0.9869%,-0.1450%,1.2224%,1.4570%,0.5367%,-0.4607%,0.5799%,-1.9919%


In [2]:
####################################
# Minimizing Portfolio Variance
####################################

import cvxpy as cv
from timeit import default_timer as timer

# Defining initial inputs
mu = Y.mean().to_numpy().reshape(1,-1)
sigma = Y.cov().to_numpy()

# Defining initial variables
x = cv.Variable((mu.shape[1], 1))

# Budget and weights constraints
constraints = [cv.sum(x) == 1,
               x >= 0]

# Defining risk objective
risk = cv.quad_form(x, sigma)
objective = cv.Minimize(risk)

# Solving problem
prob = cv.Problem(objective, constraints)
prob.solve()

# Showing Optimal Weights
weights = pd.DataFrame(x.value, index=assets, columns=['Weights'])
display(weights)

,Weights
APA,0.0004%
BA,-0.0001%
BAX,5.3794%
BMY,4.4313%
CMCSA,2.3250%
CNP,7.1112%
CPB,3.2684%
DE,0.1296%
HPQ,-0.0004%
JCI,2.9044%


As we can see the use of CVXPY's __quad_form__ in portfolio optimization can give small negative values to weights that must be zero.

In [3]:
# Calculating Portfolio Stats
var = weights.T @ Y.cov() @ weights
std = np.sqrt(var)
ret = Y.mean().to_frame().T @ weights

stats = pd.concat([ret, std, var])
stats.index = ['Return', 'Std. Dev.', 'Variance']
stats.columns = ['Values']

display(stats)

,Values
Return,0.0508%
Std. Dev.,0.6535%
Variance,0.0043%


### 1.2 Standard Deviation Minimization

An alternative problem to the quadratic problem is to minimize the standard deviation (square root of variance). To do this we can use the SOC constraint. The minimization of portfolio standard deviation can be posed as:

$$
\begin{equation}
\begin{aligned}
& \underset{x}{\text{min}} & &  g \\
& \text{s.t.} & & \mu x^{\tau} \geq \bar{\mu} \\
& & &  \sum_{i=1}^{N} x_i = 1 \\
& & & \left\|\Sigma^{1/2} x\right\| \leq g \\
& & &  x_i \geq 0 \; ; \; \forall \; i =1, \ldots, N  \\
\end{aligned}
\end{equation}
$$

Where $\left\|\Sigma^{1/2} x\right\| \leq g$ is the SOC constraint, $x$ are the weights of assets, $\mu$ is the mean vector of expected returns, $\bar{\mu}$ the minimum expected return of portfolio and $r$ is the matrix of observed returns.

__Note:__ the SOC constraint can be expressed as $(g,\Sigma^{1/2} x) \in Q^{n+1}$, this notation is used to model the SOC constraint in CVXPY.

The advantage of this formulation is that we can 

In [4]:
#########################################
# Minimizing Portfolio Standard Deviation
#########################################

from scipy.linalg import sqrtm

# Defining initial inputs
mu = Y.mean().to_numpy().reshape(1,-1)
sigma = Y.cov().to_numpy()
G = sqrtm(sigma)

# Defining initial variables
x = cv.Variable((mu.shape[1], 1))
g = cv.Variable(nonneg=True)

# Budget and weights constraints
constraints = [cv.sum(x) == 1,
               x >= 0]

# Defining risk objective
risk = g
constraints += [cv.SOC(g, G @ x)] # SOC constraint
objective = cv.Minimize(risk)

# Solving problem
prob = cv.Problem(objective, constraints)
prob.solve()

# Showing Optimal Weights
weights = pd.DataFrame(x.value, index=assets, columns=['Weights'])
display(weights)

,Weights
APA,0.0000%
BA,0.0000%
BAX,5.2632%
BMY,4.3886%
CMCSA,2.1706%
CNP,6.9869%
CPB,3.2392%
DE,0.0796%
HPQ,0.0000%
JCI,2.8375%


As we can see the use of CVXPY's __SOC constraint__ in portfolio optimization solves the error that we see when we use __quad_form__.

In [5]:
# Calculating Portfolio Stats
var = weights.T @ Y.cov() @ weights
std = np.sqrt(var)
ret = Y.mean().to_frame().T @ weights

stats = pd.concat([ret, std, var])
stats.index = ['Return', 'Std. Dev.', 'Variance']
stats.columns = ['Values']

display(stats)

,Values
Return,0.0510%
Std. Dev.,0.6535%
Variance,0.0043%


For more portfolio optimization models and applications, you can see the CVXPY based library __[Riskfolio-Lib](https://github.com/dcajasn/Riskfolio-Lib)__.

Finally, I hope you liked this example.